<a href="https://colab.research.google.com/github/YEONGHUN-H/Healthcare_project/blob/main/Data_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1차원 데이터 전처리

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 데이터셋 로드
file_path = '/content/drive/MyDrive/Colab Notebooks/기계학습프로그램/데이터/healthcare_dataset.csv'
data = pd.read_csv(file_path)

# 1단계: 결측값 확인
missing_values = data.isnull().sum()
print("각 열의 결측값:\n", missing_values)

# 결측값을 각 열의 중앙값으로 채움 (숫자형 열만)
for colums in data.select_dtypes(include=['float64', 'int64']).columns:
    if missing_values[colums] > 0:
        data[colums].fillna(data[colums].median(), inplace=True)

# 2단계: 새로운 특성 생성 - 입원 기간
data['Date of Admission'] = pd.to_datetime(data['Date of Admission'])  # 날짜 변환 (입원)
data['Discharge Date'] = pd.to_datetime(data['Discharge Date'])        # 날짜 변환 (퇴원)
data['How long stay'] = (data['Discharge Date'] - data['Date of Admission']).dt.days  # 입원 기간 계산

# 3단계: 나이를 범위로 구분
age_bins = [0, 20, 40, 60, 80, 100]  # 나이 범위
age_labels = ['0-20', '21-40', '41-60', '61-80', '81-100']  # 각 나이의 범위에 대한 라벨
data['Age Group'] = pd.cut(data['Age'], bins=age_bins, labels=age_labels, right=False)  # 나이 범주화

# 4단계: 범주형 변수 인코딩
categorical_columns = ['Gender', 'Blood Type', 'Medical Condition', 'Insurance Provider',
                       'Admission Type', 'Medication', 'Test Results']

label_encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    data[col] = label_encoders[col].fit_transform(data[col])  # 각 열을 숫자형으로 변환

# 5단계: 숫자형 열 정규화
scaler = StandardScaler()
data['Billing Amount'] = scaler.fit_transform(data[['Billing Amount']])  # 청구 금액 정규화

# 분석에 필요 없는 열 삭제
data = data.drop(columns=['Name', 'Doctor', 'Hospital', 'Room Number'])

# 6단계: 전처리된 데이터 저장
output_file_path = '/content/drive/MyDrive/Colab Notebooks/기계학습프로그램/1차원데이터전처리.csv'
data.to_csv(output_file_path, index=False)

print(f"전처리된 데이터가 {output_file_path}에 저장되었습니다.")


각 열의 결측값:
 Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64
전처리된 데이터가 /content/drive/MyDrive/Colab Notebooks/기계학습프로그램/1차원데이터전처리.csv에 저장되었습니다.


# 데이터 이미지화하는 코드.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os

# 데이터셋 로드
file_path = '/content/drive/MyDrive/Colab Notebooks/기계학습프로그램/데이터/healthcare_dataset.csv'
data = pd.read_csv(file_path)

# 1단계: 결측값 확인
missing_values = data.isnull().sum()
print("각 열의 결측값:\n", missing_values)

# 결측값을 각 열의 중앙값으로 채움 (숫자형 열만)
for colums in data.select_dtypes(include=['float64', 'int64']).columns:
    if missing_values[colums] > 0:
        data[colums].fillna(data[colums].median(), inplace=True)

# 2단계: 새로운 특성 생성 - 입원 기간
data['Date of Admission'] = pd.to_datetime(data['Date of Admission'])  # 날짜 변환 (입원)
data['Discharge Date'] = pd.to_datetime(data['Discharge Date'])        # 날짜 변환 (퇴원)
data['How long stay'] = (data['Discharge Date'] - data['Date of Admission']).dt.days  # 입원 기간 계산

# 3단계: 나이를 범위로 구분
age_bins = [0, 20, 40, 60, 80, 100]  # 나이 범위
age_labels = ['0-20', '21-40', '41-60', '61-80', '81-100']  # 각 나이의 범위에 대한 라벨
data['Age Group'] = pd.cut(data['Age'], bins=age_bins, labels=age_labels, right=False)  # 나이 범주화

# 4단계: 범주형 변수 인코딩
categorical_columns = ['Gender', 'Blood Type', 'Medical Condition', 'Insurance Provider',
                       'Admission Type', 'Medication', 'Test Results']

label_encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    data[col] = label_encoders[col].fit_transform(data[col])  # 각 열을 숫자형으로 변환

# 5단계: 숫자형 열 정규화
scaler = StandardScaler()
data['Billing Amount'] = scaler.fit_transform(data[['Billing Amount']])  # 청구 금액 정규화

# 분석에 필요 없는 열 삭제
data = data.drop(columns=['Name', 'Doctor', 'Hospital', 'Room Number'])

# 이미지화 함수 정의
def create_image_from_data(row, image_size=(28, 28)):
    """1차원 데이터를 정규화하고 이미지 형태로 변환"""
    array = np.array(row)
    normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))  # 정규화
    reshaped_array = np.resize(normalized_array, image_size)  # 이미지 형태로 변환
    return reshaped_array

# 데이터에서 숫자형 열만 선택
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
image_data = []

# 각 행을 이미지화
for idx, row in data[numerical_columns].iterrows():
    image = create_image_from_data(row, image_size=(28, 28))
    image_data.append(image.flatten())  # 이미지 데이터를 1차원으로 저장

# 이미지 데이터를 새로운 DataFrame으로 변환
image_df = pd.DataFrame(image_data)

# 이미지화된 데이터를 저장할 경로
output_image_file_path = '/content/drive/MyDrive/Colab Notebooks/기계학습프로그램/이미지화데이터.csv'

# CSV로 저장
os.makedirs(os.path.dirname(output_image_file_path), exist_ok=True)
image_df.to_csv(output_image_file_path, index=False)

print(f"이미지화된 데이터가 {output_image_file_path}에 저장되었습니다.")


각 열의 결측값:
 Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64
이미지화된 데이터가 /content/drive/MyDrive/Colab Notebooks/기계학습프로그램/이미지화데이터.csv에 저장되었습니다.
